In [5]:
import csv
import numpy as np
import scipy.stats as sps
from scipy.stats import norm
from scipy.optimize import root
import pandas as pd


np.set_printoptions(suppress=True)

file = open('bwght.csv')
file.close()

inmat = []
with open('bwght.csv') as file:
    ## Type what you want here and it works without closing the file
    reader = csv.reader(file)
    for line in reader:
        inmat.append(line)


inmat = [row[:4]+row[6:] for row in inmat]

print(inmat[0])
for ndx,item in enumerate(inmat[0]):
    print(ndx,item)
data = np.matrix(inmat[1:],dtype=float)


['faminc', 'cigtax', 'cigprice', 'bwght', 'parity', 'male', 'white', 'cigs', 'lbwght', 'bwghtlbs', 'packs', 'lfaminc']
(0, 'faminc')
(1, 'cigtax')
(2, 'cigprice')
(3, 'bwght')
(4, 'parity')
(5, 'male')
(6, 'white')
(7, 'cigs')
(8, 'lbwght')
(9, 'bwghtlbs')
(10, 'packs')
(11, 'lfaminc')


In [ ]:
smokes=np.array(data[:,7]>0,dtype=int)

In [4]:
cd "C:\Users\jap090020\Dropbox\UTDallas\buan6340\post"

C:\Users\jap090020\Dropbox\UTDallas\buan6340\post


In [37]:
def cross(*args):  # Can put either 1 or 2 things into this function
    x = args[0]
    if len(args)==1:
        y = x
    else:
        y = args[1]
    return(x.T.dot(y))
ls = lambda X,y: np.linalg.solve(cross(X),cross(X,y))
class ols:
    def __init__(self,X,y):
        self.n = X.shape[0]
        self.X = np.hstack((np.ones((self.n,1)),X))
        self.y = y
        self.b = ls(self.X,self.y)
        self.pred = self.X.dot(self.b)
        self.res = self.y-self.pred
        RSS = cross(self.res)[0,0]
        ybar = self.y.mean()
        e_0 = self.y-ybar
        TSS = cross(e_0)[0,0]
        self.rsq = 1-RSS/TSS
        sig = np.sqrt(RSS/self.n)
        self.vcov = np.linalg.inv(cross(self.X))*(sig**2)
    def predict(self,newX):
        newn = newX.shape[0]
        newX = np.hstack((np.ones((newn,1)),newX))
        return newX.dot(self.b)
    def tidy(self):
        se = np.sqrt(np.diag(self.vcov)).reshape(-1,1)
        tstats = self.b/se
        pval = norm.cdf(-np.abs(tstats))*2
        outp = np.hstack((self.b,se,tstats,pval))
        outp = pd.DataFrame(outp)
        outp.columns = ['beta','stderr','tstat','pval']
        print(outp)
        print("RSq="+str(self.rsq))    
class white(ols):
    def __init__(self,X,y):
        ols.__init__(self,X,y)
        vb = np.diagflat(np.power(self.res,2))
        vb = self.X.T.dot(vb).dot(self.X)
        vb = np.linalg.inv(cross(self.X))*vb*np.linalg.inv(cross(self.X))
        self.vcov = vb
def cross(*args):  # Can put either 1 or 2 things into this function
    x = args[0]
    if len(args)==1:
        y = x
    else:
        y = args[1]
    return(x.T.dot(y))
class ols_nocon:
    def __init__(self,X,y):
        self.n = X.shape[0]
        self.X = X
        self.y = y
        self.b = ls(self.X,self.y)
        self.pred = self.X.dot(self.b)
        self.res = self.y-self.pred
        RSS = cross(self.res)[0,0]
        ybar = self.y.mean()
        e_0 = self.y-ybar
        TSS = cross(e_0)[0,0]
        self.rsq = 1-RSS/TSS
        sig = np.sqrt(RSS/self.n)
        self.vcov = np.linalg.inv(cross(self.X))*(sig**2)
    def predict(self,newX):
        newn = newX.shape[0]
        newX = np.hstack((np.ones((newn,1)),newX))
        return newX.dot(self.b)
    def tidy(self):
        se = np.sqrt(np.diag(self.vcov)).reshape(-1,1)
        tstats = self.b/se
        pval = norm.cdf(-np.abs(tstats))*2
        outp = np.hstack((self.b,se,tstats,pval))
        outp = pd.DataFrame(outp)
        outp.columns = ['beta','stderr','tstat','pval']
        print(outp)
        print("RSq="+str(self.rsq))    

In [7]:
y = data[:,3]
y
x = data[:,7]
x
n = len(y)
ols(x,y).tidy()

         beta    stderr       tstat          pval
0  119.771900  0.571928  209.417734  0.000000e+00
1   -0.513772  0.090426   -5.681704  1.333596e-08
RSq=0.02272912110605263


In [8]:
white(x,y).tidy()

         beta    stderr       tstat          pval
0  119.771900  0.574135  208.612667  0.000000e+00
1   -0.513772  0.087670   -5.860284  4.620768e-09
RSq=0.02272912110605263


In [9]:
y = data[:,3]
x = np.hstack((data[:,7],data[:,0],data[:,5],data[:,6]))
white(x,y).tidy()

         beta    stderr      tstat          pval
0  112.065256  1.449067  77.336129  0.000000e+00
1   -0.474160  0.089281  -5.310865  1.091059e-07
2    0.060055  0.029604   2.028614  4.249763e-02
3    3.145240  1.063350   2.957859  3.097840e-03
4    5.407262  1.429363   3.782987  1.549572e-04
RSq=0.046451096834047245


In [10]:
bwght = pd.read_csv('bwght.csv')

In [11]:
x = bwght['cigs'].values.reshape(-1,1)
y = bwght['bwght'].values.reshape(-1,1)
ols(x,y).tidy()

         beta    stderr       tstat          pval
0  119.771900  0.571928  209.417734  0.000000e+00
1   -0.513772  0.090426   -5.681704  1.333596e-08
RSq=0.02272912110605263


In [12]:
x = pd.concat((bwght['cigs'],bwght['faminc']),1).values
y = bwght['bwght'].values.reshape(-1,1)
ols(x,y).tidy()

         beta    stderr       tstat          pval
0  116.974130  1.047850  111.632526  0.000000e+00
1   -0.463408  0.091478   -5.065792  4.067055e-07
2    0.092765  0.029156    3.181635  1.464461e-03
RSq=0.029804837327581435


In [18]:
def mspe(model,newX,newy):
    n = newX.shape[0]
    err = newy-model.predict(newX)
    return cross(err)[0,0]/n
def kfold(X,y,k,model=ols,stat=mspe):
    outp = np.zeros((k,))
    n = X.shape[0]
    for j in range(k):
        test = np.arange((j)*(n//k),(j+1)*(n//k))
        trainl = np.arange(0,(j)*(n//k))
        trainu = np.arange((j+1)*(n//k),n)
        train=np.concatenate((trainl,trainu))
        outp[j] = stat(model(X[train,],y[train]),X[test,],y[test])
    return outp

x = pd.concat((bwght['cigs'],bwght['faminc']),1).values
y = bwght['bwght'].values.reshape(-1,1)
ols(x,y).tidy()
kfold(x,y,10,model=white).mean()

         beta    stderr       tstat          pval
0  116.974130  1.047850  111.632526  0.000000e+00
1   -0.463408  0.091478   -5.065792  4.067055e-07
2    0.092765  0.029156    3.181635  1.464461e-03
RSq=0.029804837327581435


402.8963171909034

In [20]:
bwght['smokes'] = (bwght['cigs']>0).astype(int)
#bwght['smokes']

In [21]:
bigF = lambda x: 1/(1+np.exp(-x))
littleF = lambda x: np.multiply(bigF(x),(1-bigF(x)))
class logit:
    def __init__(self,X,y):
        self.n = X.shape[0]
        self.y = y
        self.bigX = np.ones((self.n,1))
        self.fit()
        self.nullll = self.logL()
        self.bigX = np.hstack((self.bigX,X))
        self.fit()
        self.loglike = self.logL()
        self.rsq = 1-(self.loglike/self.nullll)
    def logL(self):
        val = cross(self.y,np.log(self.pred))+cross((1-self.y),np.log(1-self.pred))
        return val[0,0]
    def deriv(self,b):
        diff=self.y-bigF(self.bigX.dot(b.reshape(-1,1)))
        jac = self.bigX.T.dot(diff)
        return np.array(jac).flatten()
    def hess(self,b):
        diff=-littleF(self.bigX.dot(b.reshape(-1,1)))
        return self.bigX.T.dot(np.diagflat(diff)).dot(self.bigX)
    def fit(self):
        soln = root(self.deriv,x0=np.zeros((self.bigX.shape[1],)),jac=self.hess)
        self.b = soln.x.reshape(-1,1)
        self.vcov = np.linalg.inv(-soln.fjac)
        self.pred = bigF(self.bigX.dot(self.b))
    def tidy(self):
        se = np.sqrt(np.diag(self.vcov)).reshape(-1,1)
        tstats = self.b/se
        pval = norm.cdf(-np.abs(tstats))*2
        outp = np.hstack((self.b,se,tstats,pval))
        outp = pd.DataFrame(outp)
        outp.columns = ['beta','stderr','tstat','pval']
        print(outp)
        print("McFadden RSq="+str(self.rsq))
    def predict(self,newX):
        newn = newX.shape[0]
        newX = np.hstack((np.ones((newn,1)),newX))
        return bigF(newX.dot(self.b))


In [22]:
x = pd.concat((bwght['faminc'],bwght['white']),1).values
y = bwght['smokes'].values.reshape(-1,1)
logit(x,y).tidy()
kfold(x,y,4,model=logit).mean()

       beta    stderr     tstat          pval
0 -1.116268  0.188049 -5.936043  2.919824e-09
1 -0.036162  0.212576 -0.170115  8.649198e-01
2  0.407269  0.930165  0.437846  6.614977e-01
McFadden RSq=0.04797995773603969


0.12535597669536236

In [23]:
x = pd.concat((bwght['cigs'],bwght['faminc'],bwght['male'],bwght['white']),1).values
y = bwght['bwght'].values.reshape(-1,1)
ols(x,y).tidy()


         beta    stderr      tstat          pval
0  112.065256  1.440059  77.819895  0.000000e+00
1   -0.474160  0.090753  -5.224703  1.744349e-07
2    0.060055  0.030392   1.976040  4.815023e-02
3    3.145240  1.068830   2.942695  3.253687e-03
4    5.407262  1.362849   3.967615  7.259560e-05
RSq=0.04645109683404747


In [33]:
sigvec = [0.25,0.5,1,2]
Tvec = [10,25,50,100,250]
rvec = [1,3,5,10]
nsim = 1000

T = 100
r = 3
sig = 0.5

outp = np.zeros((nsim,len(sigvec),len(Tvec),len(rvec)))

for sigdx,sig in enumerate(sigvec):
    for tdx,T in enumerate(Tvec):
        for rdx,r in enumerate(rvec):
            for isim in range(nsim):
                x = np.random.normal(size=(T,r),scale=sig)
                y = np.random.normal(size=(T,1),scale=sig)
                for t in range(1,T):
                    x[t,]+=x[t-1,]
                    y[t]+=y[t-1]
                outp[isim,sigdx,tdx,rdx] = ols(x,y).rsq    


In [35]:

outp2 = np.zeros((nsim,len(sigvec),len(Tvec),len(rvec)))

for sigdx,sig in enumerate(sigvec):
    for tdx,T in enumerate(Tvec):
        for rdx,r in enumerate(rvec):
            for isim in range(nsim):
                x = np.random.normal(size=(T,r),scale=sig)
                y = np.random.normal(size=(T,1),scale=sig)
                outp2[isim,sigdx,tdx,rdx] = ols(x,y).rsq    


In [34]:
outp.mean(0).mean(0)

array([[0.27148273, 0.59032351, 0.76802972, 1.        ],
       [0.24513643, 0.51841291, 0.65951954, 0.82389073],
       [0.24331313, 0.51218195, 0.64878539, 0.79246494],
       [0.23806668, 0.4995559 , 0.63603004, 0.78510479],
       [0.23974025, 0.50621497, 0.64125157, 0.78516041]])

In [36]:
outp2.mean(0).mean(0)

array([[0.11498145, 0.33480516, 0.55673224, 1.        ],
       [0.04089157, 0.12377239, 0.20680609, 0.41728546],
       [0.01996306, 0.06098981, 0.10184739, 0.20500163],
       [0.01001094, 0.03024601, 0.05035408, 0.10161372],
       [0.00401774, 0.01204961, 0.0203418 , 0.04009574]])

In [43]:
sigvec = [0.25,0.5,1,2]
Tvec = [10,25,50]
rvec = [-0.75,-0.5,-0.25,0,0.25,0.5,0.75]
nsim = 1000

T = 100
r = 3
sig = 0.5

outp = np.zeros((nsim,len(sigvec),len(Tvec),len(rvec)))
outp2 = np.zeros((nsim,len(sigvec),len(Tvec),len(rvec)))
rvals = np.zeros((nsim,len(sigvec),len(Tvec),len(rvec)))
Tvals = np.zeros((nsim,len(sigvec),len(Tvec),len(rvec)))
sigvals = np.zeros((nsim,len(sigvec),len(Tvec),len(rvec)))
for sigdx,sig in enumerate(sigvec):
    for tdx,T in enumerate(Tvec):
        for rdx,r in enumerate(rvec):
            for isim in range(nsim):
                y = np.random.normal(size=(T+10,1),scale=sig)
                for t in range(1,T+10):
                    y[t]+=r*y[t-1]
                y = y[10:]
                outp[isim,sigdx,tdx,rdx] = ols_nocon(y[:-1],y[1:]).b
                outp2[isim,sigdx,tdx,rdx] = ols(y[:-1],y[1:]).b[1]
                rvals[isim,sigdx,tdx,rdx] = r
                Tvals[isim,sigdx,tdx,rdx] = T
                sigvals[isim,sigdx,tdx,rdx] = sig
                

In [52]:
y = outp.mean(0).reshape(-1,1)-rvals.mean(0).reshape(-1,1)
x1 = Tvals.mean(0).reshape(-1,1)
x1 = 1/x1
x2 = np.multiply(x1,rvals.mean(0).reshape(-1,1))
x = np.hstack((x1,x2))
ols(x,y).tidy()

       beta    stderr      tstat      pval
0  0.000063  0.001506   0.042063  0.966448
1  0.010434  0.023817   0.438088  0.661323
2 -1.564625  0.025603 -61.111980  0.000000
RSq=0.9780039414726629


In [53]:
y = outp2.mean(0).reshape(-1,1)-rvals.mean(0).reshape(-1,1)
x1 = Tvals.mean(0).reshape(-1,1)
x1 = 1/x1
x2 = np.multiply(x1,rvals.mean(0).reshape(-1,1))
x = np.hstack((x1,x2))
ols(x,y).tidy()

       beta    stderr      tstat      pval
0  0.004342  0.001805   2.405114  0.016167
1 -1.235803  0.028546 -43.291897  0.000000
2 -2.812644  0.030686 -91.659298  0.000000
RSq=0.9918915907573435


In [71]:
from cvxopt import solvers,matrix

solvers.options['show_progress'] = False

class Lasso(ols):
    def __init__(self,x,y,T):
        ols.__init__(self,x,y)
        self.T = T
        self.bols = self.b
        self.b = self.fit(self.X,self.y)
        self.pred = self.X.dot(self.b)
        self.res = self.y-self.pred
        self.rsq = None
    def fit(self,x,y):
        r = x.shape[1]
        matP = np.matrix([[1,-1],[-1,1]])
        P = np.kron(matP,cross(x))
        matq = np.matrix([[1],[-1]])
        q = np.kron(matq,-cross(x,y))
        G = -np.eye(2*r)
        G = np.vstack((np.ones((1,2*r)),G))
        h = np.vstack((np.abs(self.bols).sum()*self.T,np.zeros((2*r,1))))
        w = solvers.qp(matrix(P),matrix(q),matrix(G),matrix(h))
        w = w['x']
        w = np.matrix(w)
        return w[:r]-w[r:]
    def kfold(self,k):
        outp = np.zeros((k,))
        for j in range(k):
            test = np.arange((j)*(self.n//k),(j+1)*(self.n//k))
            trainl = np.arange(0,(j)*(self.n//k))
            trainu = np.arange((j+1)*(self.n//k),self.n)
            train=np.concatenate((trainl,trainu))
            bstar = self.fit(self.X[train,],self.y[train])
            err = self.y[test]-self.X[test,].dot(bstar)
            outp[j] = np.power(err,2).mean()
        return outp

x = pd.concat((bwght['cigs'],bwght['faminc'],bwght['male'],bwght['white']),1).values
y = bwght['bwght'].values.reshape(-1,1)
x = x - x.mean(0)
y = y - y.mean()
print(ols(x,y).b)
Lasso(x,y,0.5).b
for i in range(101):
    print(i/100.0,Lasso(x,y,i/100.0).kfold(5).mean())


[[ 0.        ]
 [-0.47415993]
 [ 0.06005485]
 [ 3.14523963]
 [ 5.40726154]]
(0.0, 414.6471067942356)
(0.01, 410.1280838095669)
(0.02, 407.4582208938515)
(0.03, 405.3662767599605)
(0.04, 403.8522520935441)
(0.05, 402.91613998734226)
(0.06, 402.61622747507664)
(0.07, 402.5265627269838)
(0.08, 402.4199180165324)
(0.09, 402.30941546815575)
(0.1, 402.20024738212805)
(0.11, 402.07810762352085)
(0.12, 401.95766274043604)
(0.13, 401.8391501688185)
(0.14, 401.7196061603371)
(0.15, 401.60127345716893)
(0.16, 401.48470239958164)
(0.17, 401.36993223238227)
(0.18, 401.2569244434073)
(0.19, 401.1456970285682)
(0.2, 401.0362454274634)
(0.21, 400.9285727166699)
(0.22, 400.82267630100154)
(0.23, 400.71855394494355)
(0.24, 400.61621221232355)
(0.25, 400.5156417837984)
(0.26, 400.4168399949069)
(0.27, 400.3198382727537)
(0.28, 400.2232851999221)
(0.29, 400.11406722554347)
(0.3, 400.00389981098465)
(0.31, 399.89529178153305)
(0.32, 399.78825090464034)
(0.33, 399.68277753882984)
(0.34, 399.5788738197508)
(

In [67]:
Lasso(x,y,0.85).b

matrix([[ 0.        ],
        [-0.46899833],
        [ 0.06492681],
        [ 2.62354887],
        [ 4.56623455]])

In [72]:
n = 5
T = 3
np.kron(np.eye(n),np.ones((T,1)))

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [73]:
np.kron(np.ones((T,1)),np.eye(n))

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])